In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
import warnings
warnings.filterwarnings('ignore', message='Changing the shape of non-C contiguous array')

# Read the data

In [5]:
dfXtrain = pd.read_csv('preprocessed_csv/train_tree.csv', index_col='id', sep=';')
dfXtest = pd.read_csv('preprocessed_csv/test_tree.csv', index_col='id', sep=';')
dfYtrain = pd.read_csv('preprocessed_csv/y_train_tree.csv', header=None, names=['ID', 'COTIS'], sep=';')

In [6]:
dfYtrain = dfYtrain.set_index('ID')

# Preprocessing

Вынесем var14, department и subreg.

In [7]:
dropped_col_names = ['var14', 'department', 'subreg'] 

def drop_cols(df):
    return df.drop(dropped_col_names, axis=1), df[dropped_col_names]

In [8]:
train, dropped_train = drop_cols(dfXtrain)
test, dropped_test = drop_cols(dfXtest)

Добавим инфу о величине города из subreg'a

In [9]:
def add_big_city_cols(df, dropped_df):
    df['big'] = np.where(dropped_df['subreg'] % 100 == 0, 1, 0)
    df['average'] = np.where(dropped_df['subreg'] % 10 == 0, 1, 0)
    df['average'] = df['average'] - df['big']
    df['small'] = 1 - df['big'] - df['average']
    return df

In [10]:
train = add_big_city_cols(train, dropped_train)
test = add_big_city_cols(test, dropped_test)

Декодируем оставшиеся категориальные признаки

In [11]:
categorical = list(train.select_dtypes(exclude=[np.number]).columns)
categorical

['marque', 'energie_veh', 'profession', 'var6', 'var8']

In [12]:
list(test.select_dtypes(exclude=[np.number]).columns)

['marque', 'energie_veh', 'profession', 'var6', 'var8']

In [13]:
for col in categorical:
    print(col, train[col].nunique())

marque 154
energie_veh 5
profession 17
var6 5
var8 23


energie_veh и var6 с помощью get_dummies

In [14]:
small_cat = ['energie_veh', 'var6']

In [15]:
train = pd.get_dummies(train, columns=small_cat)
test = pd.get_dummies(test, columns=small_cat)

Для остальных посчитаем сглаженные средние таргета

In [16]:
big_cat = ['marque', 'profession', 'var8']

Описание для начала

In [17]:
df = pd.concat([dfYtrain.describe()] + [train[col].value_counts().describe() for col in big_cat], axis=1)
df

,COTIS,marque,profession,var8
count,300000.000000,154.000000,17.000000,23.000000
mean,346.063566,1946.168831,17647.058824,13018.000000
std,119.870510,8040.202889,29867.244501,24502.085807
min,94.780000,1.000000,393.000000,1.000000
25%,262.050000,3.000000,2610.000000,775.000000
50%,323.220000,11.500000,5829.000000,2208.000000
75%,407.500000,359.750000,13273.000000,7045.000000
max,1518.810000,73371.000000,110354.000000,91826.000000


Сглаживать будем с 500

Будем использовать среднее, 25%, 50% и 75%

Декодирование

In [18]:
class EncodeWithAggregates():

    def __init__(self, cols, y_train, train, *tests):
        self.cols = cols
        self.y_train = y_train
        self.train = train
        self.tests = tests
        self.Xs = (self.train,) + self.tests
        
        self.smooth_coef = 500
        self.miss_val = 'NAN'
        self.percentiles = [25, 50, 75]
        self.names = ['Mean'] + [str(q) for q in self.percentiles]
        self.aggs = [np.mean] + [self.percentile_fix(q) for q in self.percentiles]
        self.miss_val_fills = [agg(y_train) for agg in self.aggs]
        self.train_aggs = [agg(y_train) for agg in self.aggs]

    def percentile_fix(self, q):
        def wrapped(a):
            return np.percentile(a, q)

        return wrapped
        
    
    def transform(self):
        for col in self.cols:
            self.encode(col)
            gc.collect()
        return self.Xs
    
    
    def encode(self, col):
        df = pd.concat([self.y_train, self.train[col]], axis=1)
        dfgb = df.groupby(col)
        dfsize = dfgb.size()
        dfsize.ix[self.miss_val] = 0
        
        for name, agg, miss_val_fill, train_agg in zip(self.names, self.aggs, self.miss_val_fills, self.train_aggs):
            dfm = dfgb.agg(agg)
            dfm.ix[self.miss_val] = miss_val_fill
            for X in self.Xs:                
                agg_df = dfm.ix[X[col].fillna(self.miss_val)].set_index(X.index)[self.y_train.name]
                agg_size = dfsize.ix[X[col].fillna(self.miss_val)]
                agg_size = pd.DataFrame({'size': agg_size}).set_index(X.index)['size']
                agg_name = "{}_{}".format(col, name)
                X[agg_name] = (agg_df * agg_size + self.smooth_coef * train_agg) / (self.smooth_coef + agg_size)
        
        self.Xs = [X.drop(col, axis=1) for X in self.Xs]


In [19]:
train, test = EncodeWithAggregates(big_cat, dfYtrain['COTIS'], train, test).transform()

In [20]:
test.shape

(30000, 51)

In [21]:
train.shape

(300000, 51)

# Save routines

In [23]:
dfYtest = pd.DataFrame({'ID': dfXtest.index, 'COTIS': np.zeros(test.shape[0])})
dfYtest = dfYtest[['ID', 'COTIS']]
dfYtest.head()

,ID,COTIS
0,300001,0.0
1,300002,0.0
2,300003,0.0
3,300004,0.0
4,300005,0.0


In [24]:
def save_to_file(y, file_name):
    dfYtest['COTIS'] = y
    dfYtest.to_csv('results/{}'.format(file_name), index=False, sep=';')

# Train XGB

In [184]:
import xgboost as xgb
XGBR = xgb.XGBRegressor

In [185]:
def mape(y_true, y_pred): 
    return -np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [186]:
def mape_scorer(est, X, y):
    return mape(y, est.predict(X))

In [187]:
def log_mape_scorer(est, X, y):
    return mape(np.exp(y), np.exp(est.predict(X)))

In [65]:
kwargs = {'objective':'reg:linear', 'missing': -9999, 'seed': 56, 'n_estimators': 500}


clf = XGBR(**kwargs)
clf.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=-9999, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=56, silent=True, subsample=1)

In [68]:
y_pred = clf.predict(x_validation)
mape(y_validation, y_pred)

-16.881022061685456

Деление на train и test даёт более справедливые результаты. Теперь посмотрим на lmse

In [70]:
%%time

kwargs = {'objective':'reg:linear', 'missing': -9999, 'seed': 56, 'n_estimators': 500}


clf = XGBR(**kwargs)
clf.fit(x_train, np.log(y_train))

CPU times: user 5min 19s, sys: 1.53 s, total: 5min 21s
Wall time: 1min 23s


In [71]:
y_pred = np.exp(clf.predict(x_validation))
mape(y_validation, y_pred)

-16.324240730866656

Теперь попробуем asymmetric mse, но сначала проверим, что мы умеем запускать xgb через стандартный интерфейс

In [82]:
%%time

dtrain = xgb.DMatrix(x_train, label=y_train, missing=-9999)
dvalidation = xgb.DMatrix(x_validation, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.1, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'objective':'reg:linear', 'reg_alpha':0, 'reg_lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

bst = xgb.train(param, dtrain, num_round)

CPU times: user 2min 56s, sys: 177 ms, total: 2min 57s
Wall time: 2min 57s


In [83]:
y_pred = bst.predict(dvalidation)
mape(y_validation, y_pred)

-16.881022061685456

Норм, теперь результаты совпадают. Значения у параметров по умолчанию разные в xgb.train и в sklearn'овских обёртках

### AMSE

Опять сначала проверка

In [119]:
def amse(preds, dtrain, divider=1):
    labels = dtrain.get_label()
    grad = 2 * np.where(preds > labels, (preds - labels), (preds - labels) / divider)
    second_grad = 4 * np.where(preds > labels, 1, 1 / divider)
    return grad, second_grad

In [120]:
def amse_divider(divider):
    def wrapper(*args, **kwargs):
        return amse(*args, divider=divider, **kwargs)
    return wrapper

grad / 2, second_grad / 2

In [107]:
%%time

dtrain = xgb.DMatrix(x_train, label=y_train, missing=-9999)
dvalidation = xgb.DMatrix(x_validation, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.1, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'alpha':0, 'lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

bst = xgb.train(param, dtrain, num_round, obj=amse_divider(1))

CPU times: user 4min 31s, sys: 3.62 s, total: 4min 35s
Wall time: 4min 35s


In [108]:
y_pred = bst.predict(dvalidation)
mape(y_validation, y_pred)

-16.881022061685456

Ок, всё работает (если поделить на два настоящий mse, как и реализованно в xgboost). Теперь немного поправим mse, увеличим градиент в два раза (оставив настоящий), а второй градиент в четыре (в два раза больше настоящего) и уменьше eta.

In [115]:
%%time

dtrain = xgb.DMatrix(x_train, label=y_train, missing=-9999)
dvalidation = xgb.DMatrix(x_validation, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.05, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'alpha':0, 'lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

bst = xgb.train(param, dtrain, num_round, obj=amse_divider(1))

CPU times: user 4min 34s, sys: 3.49 s, total: 4min 37s
Wall time: 4min 37s


In [116]:
y_pred = bst.predict(dvalidation)
mape(y_validation, y_pred)

-11.156682332738244

AMSE

In [125]:
%%time

dtrain = xgb.DMatrix(x_train, label=y_train, missing=-9999)
dvalidation = xgb.DMatrix(x_validation, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.05, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'alpha':0, 'lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

divider = 2

bst = xgb.train(param, dtrain, num_round, obj=amse_divider(divider))

CPU times: user 4min 31s, sys: 3.38 s, total: 4min 35s
Wall time: 4min 35s


In [126]:
y_pred = bst.predict(dvalidation)
mape(y_validation, y_pred)

-9.578534964952782

# Save

In [128]:
%%time

dtrain = xgb.DMatrix(x_bigtrain, label=y_bigtrain, missing=-9999)
dtest = xgb.DMatrix(x_test, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.05, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'alpha':0, 'lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

divider = 2

bst = xgb.train(param, dtrain, num_round, obj=amse_divider(divider))

CPU times: user 4min 42s, sys: 3.86 s, total: 4min 46s
Wall time: 4min 48s


In [136]:
y_pred = bst.predict(dtrain)
mape(y_bigtrain, y_pred)

-10.73870528204605

In [137]:
%%time

dtrain = xgb.DMatrix(x_train, label=y_train, missing=-9999)
dvalidation = xgb.DMatrix(x_validation, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.05, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'alpha':0, 'lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

divider = 2

bst = xgb.train(param, dtrain, num_round, obj=amse_divider(divider))

CPU times: user 4min 59s, sys: 3.86 s, total: 5min 3s
Wall time: 5min 3s


In [138]:
y_pred = bst.predict(dtrain)
mape(y_train, y_pred)

-10.746545997142359

In [139]:
y_predict = bst.predict(dtest)

In [141]:
def amse(preds, dtrain, divider=1):
    labels = dtrain.get_label()
    grad = 1 * np.where(preds > labels, (preds - labels), (preds - labels) / divider)
    second_grad = 1 * np.where(preds > labels, 1, 1 / divider)
    return grad, second_grad

In [142]:
def amse_divider(divider):
    def wrapper(*args, **kwargs):
        return amse(*args, divider=divider, **kwargs)
    return wrapper

In [143]:
%%time

dtrain = xgb.DMatrix(x_bigtrain, label=y_bigtrain, missing=-9999)
dtest = xgb.DMatrix(x_test, missing=-9999)

param =   {'base_score':0.5, 'colsample_bylevel':1, 'colsample_bytree':1, 'gamma':0,
           'eta':0.1, 'max_delta_step':0, 'max_depth':3,
           'min_child_weight':1, 'nthread':-1,
           'alpha':0, 'lambda':1,
           'scale_pos_weight':1, 'seed':56, 'silent':True, 'subsample':1}
num_round = 500

divider = 2

bst = xgb.train(param, dtrain, num_round, obj=amse_divider(divider))

CPU times: user 4min 40s, sys: 3.75 s, total: 4min 44s
Wall time: 4min 45s


In [145]:
y_pred = bst.predict(dtrain)
mape(y_bigtrain, y_pred)

-9.578671529090105

In [146]:
y_predict = bst.predict(dtest)

In [147]:
save_to_file(y_predict, 'xbg_500_amse.csv')